### 首先，落俗套地讓我們引入該用到的東西們，並且很勇敢的說出自己是複製貼上來的。

In [1]:
%env KERAS_BACKEND=tensorflow

env: KERAS_BACKEND=tensorflow


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact, IntSlider, Button

In [4]:
# Keras functions
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

# Keras dataset
from keras.datasets import mnist

# Keras utils
from keras.utils import np_utils

Using TensorFlow backend.


### 接著，我也不知道有甚麼資料好用的，就用minst吧^.^，順便把資料調整好

In [5]:
(x_train0, y_train0), (x_test0, y_test0) = mnist.load_data()

11493376/11490434 [==============================] - 18s 2us/step


In [9]:
#print("There are %d training data with size %d x %d" %x_train0.shape)
#print("There are %d testing  data with size %d x %d" %x_test0.shape)

x_train = x_train0.reshape(60000, 28*28)
x_test = x_test0.reshape(10000, 28*28)

x_train -= x_train.min()
x_train = x_train/x_train.max()

y_train = np_utils.to_categorical(y_train0, 10)
y_test = np_utils.to_categorical(y_test0, 10)

東西都設定好了，就讓我們來玩玩可愛的(?)function API吧! <br>
首先要決定好我們的神經網路該長甚麼樣子，總之就隨便畫畫吧XD

![my NN](https://i.imgur.com/7wlfaaE.png)

順便把函數名字跟每層的變數都取好名字了，至於每個地方的activation function...隨便啦，先都用sigmoid，有分支的就上面用relu，最後再用softmax吧<br>
之所以會想這樣拉，主要是想試試看把不同分支的部分牽在一起會怎麼樣(就是不想畫成圖論的tree就對了拉)

In [10]:
from keras.models import Model
from keras.layers import Input
from keras.layers import concatenate, add

In [11]:
f_1 = Dense(500, activation='sigmoid')
f_2 = Dense(300, activation='relu')
f_3 = Dense(300, activation='sigmoid')
f_4 = Dense(150, activation='relu')
f_5 = Dense(100, activation='sigmoid')
f_6 = Dense(350, activation='softmax')
f_7 = Dense(10, activation='softmax')

In [13]:
x = Input(shape=(784,))

h_1 = f_1(x)
h_21 = f_2(h_1)
h_22 = f_3(h_1)
h_31 = f_4(h_21)
h_32 = f_5(h_21)

u_1 = concatenate([h_32, h_22])
h_4 = f_6(u_1)

u_2 = concatenate([h_31, h_4])
y = f_7(u_2)

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
#print(x)
#print(h_1)
#print(h_21)
#print(h_22)
#print(h_31)
#print(h_32)
#print(h_4)
#print(y)

Tensor("input_2:0", shape=(?, 784), dtype=float32)
Tensor("dense_1/Sigmoid:0", shape=(?, 500), dtype=float32)
Tensor("dense_2/Relu:0", shape=(?, 300), dtype=float32)
Tensor("dense_3/Sigmoid:0", shape=(?, 300), dtype=float32)
Tensor("dense_4/Relu:0", shape=(?, 150), dtype=float32)
Tensor("dense_5/Sigmoid:0", shape=(?, 100), dtype=float32)
Tensor("dense_6/Softmax:0", shape=(?, 350), dtype=float32)
Tensor("dense_7/Softmax:0", shape=(?, 10), dtype=float32)


大功告成，可以睡...欸不是，建model然後compile啦

In [15]:
model = Model(x, y)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 784)          0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 500)          392500      input_2[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 300)          150300      dense_1[0][0]                    
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 100)          30100       dense_2[0][0]                    
__________________________________________________________________________________________________
dense_3 (D

In [19]:
model.compile(loss = 'mse', optimizer = SGD(lr=0.03), metrics=['accuracy'])

In [20]:
model.fit(x_train, y_train, batch_size = 300, epochs = 25)

Epoch 1/25
60000/60000 [==============================] - 8s 137us/step - loss: 0.0842 - acc: 0.2926
Epoch 2/25
60000/60000 [==============================] - 8s 139us/step - loss: 0.0836 - acc: 0.2959
Epoch 3/25
60000/60000 [==============================] - 8s 139us/step - loss: 0.0829 - acc: 0.2982
Epoch 4/25
60000/60000 [==============================] - 8s 141us/step - loss: 0.0821 - acc: 0.3037
Epoch 5/25
60000/60000 [==============================] - 7s 122us/step - loss: 0.0812 - acc: 0.3148
Epoch 6/25
60000/60000 [==============================] - 7s 119us/step - loss: 0.0802 - acc: 0.3253
Epoch 7/25
60000/60000 [==============================] - 7s 122us/step - loss: 0.0791 - acc: 0.3386
Epoch 8/25
60000/60000 [==============================] - 7s 120us/step - loss: 0.0780 - acc: 0.3550
Epoch 9/25
60000/60000 [==============================] - 8s 132us/step - loss: 0.0769 - acc: 0.3717
Epoch 10/25
60000/60000 [==============================] - 7s 125us/step - loss: 0.0757 - a

In [23]:
score = model.evaluate(x_test, y_test)
print("Loss: %f" %score[0])
print("準確率: %f" %(score[1]*100))

10000/10000 [==============================] - 1s 113us/step
Loss: 0.059601
準確率: 53.830000


### 哼...準確率雖然有1.8個教授，但說到底也只是54%，機器學習的世界真是奧妙呢....
不過說起來，感覺準確率還能再提升呢，讓他多做個幾倍看看吧

In [24]:
model.fit(x_train, y_train, batch_size = 300, epochs = 70)

Epoch 1/70
60000/60000 [==============================] - 7s 116us/step - loss: 0.0558 - acc: 0.6065
Epoch 2/70
60000/60000 [==============================] - 7s 119us/step - loss: 0.0545 - acc: 0.6226
Epoch 3/70
60000/60000 [==============================] - 7s 114us/step - loss: 0.0532 - acc: 0.6335
Epoch 4/70
60000/60000 [==============================] - 7s 124us/step - loss: 0.0519 - acc: 0.6477
Epoch 5/70
60000/60000 [==============================] - 8s 132us/step - loss: 0.0507 - acc: 0.6638
Epoch 6/70
60000/60000 [==============================] - 10s 159us/step - loss: 0.0495 - acc: 0.6784
Epoch 7/70
60000/60000 [==============================] - 8s 126us/step - loss: 0.0483 - acc: 0.6940
Epoch 8/70
60000/60000 [==============================] - 7s 115us/step - loss: 0.0471 - acc: 0.7107
Epoch 9/70
60000/60000 [==============================] - 7s 123us/step - loss: 0.0459 - acc: 0.7231
Epoch 10/70
60000/60000 [==============================] - 7s 119us/step - loss: 0.0448 - 

In [25]:
score = model.evaluate(x_test, y_test)
print("Loss: %f" %score[0])
print("準確率: %f" %(score[1]*100))

10000/10000 [==============================] - 1s 116us/step
Loss: 0.025542
準確率: 83.860000


### 喔喔喔提升到了84%，將近三個教授呢! 果然是我太沒耐心了。
不過看起來稍微弄亂網路並沒有很有效的提升，人生真難呢...